### LIBRERÍAS

In [1]:
import pandas as pd
import mysql.connector

### PROCESAMIENTO

In [4]:
df_ratin_final = pd.read_csv("../proyecto2/Data/rating_final.csv")
df_accepts = pd.read_csv("../proyecto2/Data/chefmozaccepts.csv")
df_cousine = pd.read_csv("../proyecto2/Data/chefmozcuisine.csv")
df_houors4 = pd.read_csv("../proyecto2/Data/chefmozhours4.csv")
df_geoplaces = pd.read_csv("../proyecto2/Data/geoplaces2.csv")
df_parking = pd.read_csv("../proyecto2/Data/chefmozparking.csv")

In [ ]:
lista_id = df_geoplaces["placeID"].unique()

df_accepts = df_accepts[df_accepts['placeID'].isin(lista_id)]
df_ratin_final = df_ratin_final[df_ratin_final['placeID'].isin(lista_id)]
df_cousine = df_cousine[df_cousine['placeID'].isin(lista_id)]
df_houors4 = df_houors4[df_houors4['placeID'].isin(lista_id)]
df_parking = df_parking[df_parking['placeID'].isin(lista_id)]

df_ratin_final.drop("userID", axis=1, inplace=True)

### FUNCIONES SQL (faltaría la contraseña de cada uno)

In [2]:
# ESTABLECER CONEXIÓN CON LA BASE DE DATOS DE MYSQL
# PARA GUARDAR LA CONEXIÓN Y EL CURSOR, HAY QUE IGUALAR LA FUNCIÓN A DOS VARIABLES
def conexion():
    db = mysql.connector.connect(host     = "localhost", # donde se aloja la base de datos
                             user     = "root", # poned vuestro usuario
                             password = "", # poned vuestra contraseña
                             database = None)
    cursor = db.cursor()
    return db, cursor

# CERRAR LA CONEXIÓN
def cerrar_conexion(cursor, db):
    cursor.fetchall() # se utiliza para recuperar todas las filas del conjunto de resultados después de ejecutar una consulta en una base de datos. Esto es útil cuando deseas trabajar con los resultados de una consulta en tu programa Python
    cursor.close()
    db.close()

# CREAR UNA BASE DE DATOS DANDOLE EL NOMBRE DE LA BASE DE DATOS
def crear_database(database):
    db, cursor = conexion()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database};")
    cerrar_conexion(cursor, db)

# CREAR UNA TABLA EN UNA BASE DE DATOS.
# PASANDOLE LA BASE DE DATOS DONDE QUEREMOS CREARLA Y LA QUERY DE LA TABLA QUE QUEREMOS CREAR
def crear_tablas(query, database):
    db, cursor = conexion()
    query_database = f"USE {database};"
    cursor.execute(query_database)
    cursor.execute(query)
    cerrar_conexion(cursor, db)

# EJECUTAR QUERYS DE CONSULTAS:
def ejecutar_consulta(database, query):
    db, cursor = conexion()
    cursor.execute(f"USE {database};")

    # Ejecutar la consulta
    cursor.execute(query)
    columnas = cursor.column_names
    # Obtener los resultados, si es una consulta SELECT
    resultados = cursor.fetchall()
    # Hacer commit para guardar los cambios (en caso de INSERT, UPDATE, DELETE, etc.)
    db.commit()
    # Cerrar la conexión
    cerrar_conexion(cursor, db)
    return resultados, columnas

def insertar_datos_porfila(df, database, tabla):
    db, cursor = conexion()
    cursor.execute(f"USE {database};")

    # Iterar sobre las filas del DataFrame e insertar cada fila en la tabla
    for _, fila in df.iterrows(): #Este método de pandas retorna un generador que produce pares (índice, fila) para cada fila del DataFrame df. 
                                  #el guion bajo _ se utiliza para descartar el índice, ya que no se está utilizando dentro del bucle.
                                  #Aquí el isinstance verifica que le valor es un str, sino lo convierte a str ya que la query tiene que ser str
        valores = ", ".join([f"'{valor}'" if isinstance(valor, str) else str(valor) for valor in fila])
        query = f"INSERT INTO {tabla} VALUES ({valores});"
        cursor.execute(query)

    # Hacer commit para guardar los cambios
    db.commit()
    cerrar_conexion(cursor, db)

### CREACIÓN DATABASE

In [7]:
crear_database("recomendador")

### QUERYS

In [8]:
query_geoplaces = """CREATE TABLE IF NOT EXISTS geoplaces (
                    placeID INT UNSIGNED NOT NULL,
                    latitude FLOAT, 
                    longitude FLOAT, 
                    the_geom_meter VARCHAR(255), 
                    name VARCHAR(255), 
                    address VARCHAR(255),
                    city VARCHAR(255), 
                    state VARCHAR(255), 
                    country VARCHAR(255), 
                    fax VARCHAR(255), 
                    zip VARCHAR(255), 
                    alcohol VARCHAR(255), 
                    smoking_area VARCHAR(255),
                    dress_code VARCHAR(255), 
                    accessibility VARCHAR(255), 
                    price VARCHAR(255), 
                    url VARCHAR(255), 
                    Rambience VARCHAR(255), 
                    franchise VARCHAR(255),
                    area VARCHAR(255), 
                    other_services VARCHAR(255),
                    PRIMARY KEY (placeID)
                    );"""

query_payments = """CREATE TABLE IF NOT EXISTS payments (
                    placeID INT UNSIGNED NOT NULL,
                    Rpayment VARCHAR(255),
                    FOREIGN KEY (placeID) REFERENCES geoplaces (placeID)
                    );"""

query_cocina = """CREATE TABLE IF NOT EXISTS cocina (
                    placeID INT UNSIGNED NOT NULL,
                    Rcuisine VARCHAR(255),
                    FOREIGN KEY (placeID) REFERENCES geoplaces (placeID)
                    );"""

query_horario = """CREATE TABLE IF NOT EXISTS horario (
                    placeID INT UNSIGNED NOT NULL,
                    hours VARCHAR(255),
                    days VARCHAR(255),
                    FOREIGN KEY (placeID) REFERENCES geoplaces (placeID)
                    );"""

query_parking = """CREATE TABLE IF NOT EXISTS parking (
                    placeID INT UNSIGNED NOT NULL,
                    parking_lot VARCHAR(255),
                    FOREIGN KEY (placeID) REFERENCES geoplaces (placeID)
                    );"""

query_rating = """CREATE TABLE IF NOT EXISTS rating (
                    placeID INT UNSIGNED NOT NULL,
                    rating INT UNSIGNED NOT NULL,
                    food_rating INT UNSIGNED NOT NULL,
                    service_rating INT UNSIGNED NOT NULL,
                    FOREIGN KEY (placeID) REFERENCES geoplaces (placeID)
                    );"""

### CREACIÓN TABLAS

In [11]:
crear_tablas(query_geoplaces, "recomendador")
crear_tablas(query_payments, "recomendador")
crear_tablas(query_cocina, "recomendador")
crear_tablas(query_horario, "recomendador")
crear_tablas(query_parking, "recomendador")
crear_tablas(query_rating, "recomendador")

### INSERCCIÓN DATOS

In [14]:
insertar_datos_porfila(df_geoplaces, "recomendador", "geoplaces")
insertar_datos_porfila(df_accepts, "recomendador", "payments")
insertar_datos_porfila(df_cousine, "recomendador", "cocina")
insertar_datos_porfila(df_houors4, "recomendador", "horario")
insertar_datos_porfila(df_parking, "recomendador", "parking")
insertar_datos_porfila(df_ratin_final, "recomendador", "rating")

### EXTRACCIÓN DATOS

In [3]:
query_select = """SELECT * FROM geoplaces
          JOIN payments ON geoplaces.placeID=payments.placeID
          JOIN cocina ON geoplaces.placeID=cocina.placeID
          JOIN horario ON geoplaces.placeID=horario.placeID
          JOIN parking ON geoplaces.placeID=parking.placeID
          ORDER BY geoplaces.placeID;
        """

In [4]:
resultado, columnas = ejecutar_consulta("recomendador", query_select)

In [5]:
df = pd.DataFrame(resultado, columns = columnas)

In [6]:
# Eliminar columnas duplicadas
df = df.loc[:, ~df.columns.duplicated()]

In [7]:
df = df[["name", "alcohol", "dress_code", "accessibility", "smoking_area", "price", "Rambience", 'area', 'other_services', 'Rpayment', 'Rcuisine', 'hours', 'days', 'parking_lot']]

In [8]:
df.head()

,name,alcohol,dress_code,accessibility,smoking_area,price,Rambience,area,other_services,Rpayment,Rcuisine,hours,days,parking_lot
0,puesto de gorditas,No_Alcohol_Served,informal,no_accessibility,permitted,low,familiar,open,none,cash,Regional,08:00-12:00;,Mon;Tue;Wed;Thu;Fri;,public
1,puesto de gorditas,No_Alcohol_Served,informal,no_accessibility,permitted,low,familiar,open,none,cash,Regional,00:00-00:00;,Sat;,public
2,puesto de gorditas,No_Alcohol_Served,informal,no_accessibility,permitted,low,familiar,open,none,cash,Regional,00:00-00:00;,Sun;,public
3,Cafe Chaires,No_Alcohol_Served,informal,completely,not permitted,low,familiar,closed,none,cash,Cafeteria,00:00-23:30;,Mon;Tue;Wed;Thu;Fri;,yes
4,Cafe Chaires,No_Alcohol_Served,informal,completely,not permitted,low,familiar,closed,none,cash,Cafeteria,00:00-23:30;,Sat;,yes


--------------------------------------------- APARTIR DE AQUÍ EMPEZAMOS ----------------------------------------

In [98]:
df["Mon;Tue;Wed;Thu;Fri;"] = df[df["days"] == "Mon;Tue;Wed;Thu;Fri;"]["hours"]
df["Sat;"] = df[df["days"] == "Sat;"]["hours"]
df["Sun;"] = df[df["days"] == "Sun;"]["hours"]

In [99]:
# Ejemplo usuario

data = [["puesto de gorditas", 1],
        ["Paniroles", 1],
        ["Pollo_Frito_Buenos_Aires", 0]
        ]
df_ana = pd.DataFrame(data = data, columns = ["name", "rating"])

In [100]:
# Buscando características de los restaurantes en los que el usuario ha estado
df_busqueda = pd.DataFrame()
for restaurante in df_ana.name:
    df_busqueda = pd.concat([df_busqueda, df[(df["name"] == restaurante)]], ignore_index=True)